In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.12.0


In [2]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
filepath = "merged_u1-50_w001_resampled.csv"
df_original = pd.read_csv(filepath)

In [4]:
df_original

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,1
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,1
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,1
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,1
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,1
...,...,...,...,...,...,...,...,...
2055043,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,50
2055044,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,50
2055045,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,50
2055046,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,50


In [5]:
df_original.isnull().sum()

timestamp    0
LAx          0
LAy          0
LAz          0
GYx          0
GYy          0
GYz          0
name         0
dtype: int64

In [7]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2055048 entries, 0 to 2055047
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   LAx        float64
 2   LAy        float64
 3   LAz        float64
 4   GYx        float64
 5   GYy        float64
 6   GYz        float64
 7   name       int64  
dtypes: float64(6), int64(2)
memory usage: 125.4 MB


In [8]:
df_original['name'].value_counts()

18    82714
47    65984
29    60052
8     60036
6     60005
9     59986
13    59984
41    59984
39    59958
3     59958
37    55050
32    52339
12    51686
2     48001
7     47947
11    47946
34    47824
49    47614
42    41946
19    41946
40    41946
35    41945
33    41945
27    41535
30    40959
24    35980
16    35974
20    35948
28    35947
31    35947
4     35946
23    34545
38    32186
44    31414
43    30100
17    29997
48    29990
50    29988
14    29950
46    29945
10    28892
1     26790
45    26022
25    25926
5     25792
22    25598
36    25173
26    23949
15    17985
21    15774
Name: name, dtype: int64

In [9]:
df_Xtime = df_original.copy()
df_Xtime = df_Xtime.drop(['timestamp'], axis = 1).copy()
df_Xtime.shape

(2055048, 7)

In [11]:
X = df_Xtime[['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz']].copy()
y = df_Xtime['name'].copy()

In [14]:
scaler = StandardScaler()
X_transformed = X.copy()
X_transformed = scaler.fit_transform(X_transformed)

scaled_X = pd.DataFrame(data = X_transformed, columns = ['LAx', 'LAy', 'LAz','GYx', 'GYy', 'GYz'])
scaled_X['label'] = y.values

scaled_X

,LAx,LAy,LAz,GYx,GYy,GYz,label
0,0.359745,-0.067021,-0.237446,0.105221,0.035341,0.175545,1
1,0.322625,-0.073783,-0.418039,0.133911,0.092852,0.143484,1
2,0.262201,-0.074915,-0.429812,0.126737,0.047108,0.117177,1
3,0.250972,-0.083358,-0.396125,0.066811,0.002223,0.092793,1
4,0.238900,-0.096951,-0.349979,-0.038693,-0.039614,0.085733,1
...,...,...,...,...,...,...,...
2055043,-2.816791,-1.777656,1.239172,-1.987696,-0.047543,-1.034070,50
2055044,-2.799397,-1.904341,0.898730,-1.852644,0.439278,-0.991908,50
2055045,-2.474708,-1.686826,0.552660,-1.745327,0.912402,-0.999240,50
2055046,-2.248585,-1.399994,0.468253,-1.707947,1.300863,-1.050568,50


In [15]:
import scipy.stats as stats

Fs = 100
frame_size = Fs*6 
hop_size = Fs*6

In [17]:
def get_frames(df, frame_size, hop_size):
    N_FEATURES = 6

    frames = []
    labels = []
    
    last_value = df['label'].iloc[-1]
    for j in range (0,last_value+1): 
        filtered_df = df[df['label'] == j]
        filtered_df = filtered_df.drop('label', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            LAx = filtered_df['LAx'].values[i: i + frame_size]
            LAy = filtered_df['LAy'].values[i: i + frame_size]
            LAz = filtered_df['LAz'].values[i: i + frame_size]
            GYx = filtered_df['GYx'].values[i: i + frame_size]
            GYy = filtered_df['GYy'].values[i: i + frame_size]
            GYz = filtered_df['GYz'].values[i: i + frame_size]
            

            label = j

            frames.append([LAx, LAy, LAz, GYx, GYy , GYz])
            labels.append(label)

    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)
    
    return frames, labels

In [18]:
X, y = get_frames(scaled_X,frame_size, hop_size)
X.shape, y.shape

((3391, 600, 6), (3391,))

In [46]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

X_train, X_validation, y_train, y_validation = train_test_split(X_train_val, y_train_val, test_size = 0.1, random_state = 0, stratify = y_train_val)
X_train.shape, X_test.shape, X_validation.shape

((2135, 600, 6), (1018, 600, 6), (238, 600, 6))

In [20]:
X_train[0].shape, X_test[0].shape

((600, 6), (600, 6))

In [47]:
train_shape_0 = X_train.shape[0]
test_shape_0 = X_test.shape[0]

train_shape_1 = X_train.shape[1]
test_shape_1 = X_test.shape[1]

validation_shape_0 = X_validation.shape[0]
validation_shape_1 = X_validation.shape[1]


X_train = X_train.reshape(train_shape_0, train_shape_1, 6, 1)
X_test = X_test.reshape(test_shape_0, test_shape_1, 6, 1)
X_validation = X_validation.reshape(validation_shape_0, validation_shape_1, 6, 1)



X_train[0].shape, X_test[0].shape, X_validation[0].shape

((600, 6, 1), (600, 6, 1), (600, 6, 1))

In [38]:
num = df_original['name'].iloc[-1]
num 

50

In [48]:
# Subtract 1 from each label in y_train
y_train = np.array(y_train) - 1
y_test = np.array(y_test) - 1
y_validation = np.array(y_validation) - 1

In [65]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation = 'relu', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2, 2), activation='relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(num, activation='softmax'))

In [66]:
model.summary(line_length = 75)

Model: "sequential_7"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d_14 (Conv2D)              (None, 599, 5, 16)            80          
                                                                           
 dropout_21 (Dropout)            (None, 599, 5, 16)            0           
                                                                           
 conv2d_15 (Conv2D)              (None, 598, 4, 32)            2080        
                                                                           
 dropout_22 (Dropout)            (None, 598, 4, 32)            0           
                                                                           
 flatten_7 (Flatten)             (None, 76544)                 0           
                                                                           
 dense_14 (Dense)                (None, 64)                    489

In [67]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

In [68]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 2,
    verbose = 0,
    restore_best_weights = True
)

In [69]:
history = model.fit(X_train, y_train, epochs = 30,callbacks = [early_stopping], 
                    validation_data= (X_validation, y_validation), verbose=1)

Epoch 1/30
67/67 [==============================] - 8s 107ms/step - loss: 4.0101 - accuracy: 0.0454 - val_loss: 3.8510 - val_accuracy: 0.0798
Epoch 2/30
67/67 [==============================] - 7s 108ms/step - loss: 3.8254 - accuracy: 0.0567 - val_loss: 3.6571 - val_accuracy: 0.0966
Epoch 3/30
67/67 [==============================] - 7s 107ms/step - loss: 3.6813 - accuracy: 0.0754 - val_loss: 3.3767 - val_accuracy: 0.2101
Epoch 4/30
67/67 [==============================] - 7s 106ms/step - loss: 3.3512 - accuracy: 0.1274 - val_loss: 2.9518 - val_accuracy: 0.3067
Epoch 5/30
67/67 [==============================] - 7s 110ms/step - loss: 3.0907 - accuracy: 0.1607 - val_loss: 2.4887 - val_accuracy: 0.3739
Epoch 6/30
67/67 [==============================] - 8s 112ms/step - loss: 2.7273 - accuracy: 0.2314 - val_loss: 2.1425 - val_accuracy: 0.4790
Epoch 7/30
67/67 [==============================] - 7s 107ms/step - loss: 2.3715 - accuracy: 0.2988 - val_loss: 1.7130 - val_accuracy: 0.5588
Epoch 

In [70]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100))

32/32 [==============================] - 1s 37ms/step - loss: 0.6971 - accuracy: 0.7741
Test loss:  0.6971. Test accuracy:  77.41%
